In [70]:
import pandas as pd

df = pd.read_csv('../coords_data/interpolated_coords.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['val'] = [0 for i in range(len(df['Datetime']))]
df

,Datetime,Lat,Long,val
0,2024-06-26 14:20:00,62.946870,-82.029900,0
1,2024-06-26 14:20:01,62.946870,-82.029900,0
2,2024-06-26 14:20:02,62.946870,-82.029900,0
3,2024-06-26 14:20:03,62.946870,-82.029900,0
4,2024-06-26 14:20:04,62.946870,-82.029900,0
...,...,...,...,...
1039796,2024-07-08 15:09:56,63.127382,-82.325561,0
1039797,2024-07-08 15:09:57,63.127381,-82.325566,0
1039798,2024-07-08 15:09:58,63.127381,-82.325571,0
1039799,2024-07-08 15:09:59,63.127381,-82.325575,0


In [71]:
# Combine interpolated data with coords data
df_data = pd.read_csv('combined.csv')
df_data['Datetime'] = pd.to_datetime(df_data['time'])
df_data = df_data.drop(columns=['Unnamed: 0', 'time'])
df = pd.merge(df_data, df, on="Datetime")
df

,bottom_hardness,bottom_roughness,wave_depth,depth,nasc0,fish_depth0,nasc1,fish_depth1,nasc2,fish_depth2,nasc3,fish_depth3,Datetime,Lat,Long,val
0,-60.51,0.0,2.1,150.6,75.97,118.1,68.85,121.3,52.31,126.7,53.76,126.4,2024-06-26 20:46:10,62.947980,-82.031340,0
1,-61.16,0.0,2.1,150.6,70.10,115.3,63.27,118.1,45.54,126.1,46.67,125.8,2024-06-26 20:46:14,62.947980,-82.031340,0
2,-61.05,0.0,2.1,150.6,84.39,115.4,76.96,118.4,53.72,126.6,55.18,126.3,2024-06-26 20:46:16,62.947980,-82.031340,0
3,-55.47,0.0,2.1,150.6,91.19,99.2,83.91,112.1,48.22,126.4,49.46,126.3,2024-06-26 20:46:18,62.947980,-82.031340,0
4,-56.92,0.0,2.1,150.6,100.78,109.6,93.15,114.0,55.86,126.8,57.40,126.5,2024-06-26 20:46:20,62.947980,-82.031340,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202850,-9.60,0.2,2.1,59.7,16.59,18.2,14.16,14.2,2.43,52.6,0.00,0.0,2024-07-06 14:17:00,62.993860,-82.024360,0
202851,-7.59,0.2,2.1,59.6,29.25,27.7,16.54,12.9,12.71,58.3,0.00,0.0,2024-07-06 14:17:02,62.993848,-82.024353,0
202852,-9.30,0.2,4.8,59.5,22.44,32.5,11.71,21.2,10.73,58.2,0.00,0.0,2024-07-06 14:17:06,62.993823,-82.024338,0
202853,-6.02,0.2,2.7,59.4,43.13,16.8,29.54,6.9,13.58,57.8,0.00,0.0,2024-07-06 14:17:08,62.993811,-82.024331,0


In [72]:
# Create H3 hexagons

import h3

h3_res = []

res = 8 # set hex resolution (size of hexagons) default is 5 for us

for longitude, latitude in zip(df['Long'].to_list(), df['Lat'].to_list()):
    h3_index = h3.geo_to_h3(
        lat=latitude,
        lng=longitude,
        resolution=res
    )
    h3_res.append(h3_index)

df['h3_res'] = h3_res

In [73]:
h3_df = df[['h3_res']] # new df with only h3 cell and id
df_points = (h3_df
             .groupby(['h3_res'])
             .median('nasc0').reset_index())
df_points

,h3_res
0,880e320001fffff
1,880e320003fffff
2,880e320005fffff
3,880e320007fffff
4,880e320009fffff
...,...
375,880e327763fffff
376,880e327765fffff
377,880e327767fffff
378,880e327769fffff


In [74]:
# Add columns from original df to the one with H3s
cols = df.columns.to_list()
cols.remove('h3_res')

for col in cols: 
    df_points[col] = df[col].groupby(df['h3_res']).median().values

df_points

,h3_res,bottom_hardness,bottom_roughness,wave_depth,depth,nasc0,fish_depth0,nasc1,fish_depth1,nasc2,fish_depth2,nasc3,fish_depth3,Datetime,Lat,Long,val
0,880e320001fffff,-22.700,1.0,4.50,133.9,82.120,57.00,27.920,16.00,47.970,60.60,11.520,112.20,2024-07-05 12:34:30,63.286195,-81.653073,0.0
1,880e320003fffff,-22.255,1.2,5.20,128.5,116.355,48.90,29.730,17.60,76.950,50.20,10.425,108.50,2024-07-05 12:14:08,63.279309,-81.659587,0.0
2,880e320005fffff,-49.225,0.0,4.50,150.6,77.850,55.65,32.815,22.05,39.600,65.00,26.750,104.40,2024-07-05 14:37:01,63.290802,-81.681289,0.0
3,880e320007fffff,-22.460,0.9,6.00,127.2,102.880,46.60,31.040,17.50,69.190,51.70,8.780,113.00,2024-07-05 12:04:06,63.275991,-81.663563,0.0
4,880e320009fffff,-23.960,0.9,4.80,137.8,72.820,58.00,27.480,20.30,44.690,60.00,16.380,111.60,2024-07-05 12:54:12,63.293126,-81.647536,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,880e327763fffff,-59.395,0.0,2.10,150.6,89.810,44.85,46.785,16.80,48.455,60.45,20.420,122.00,2024-06-29 06:53:37,63.042944,-82.004187,0.0
376,880e327765fffff,-56.870,0.0,2.70,150.6,91.015,48.40,33.320,113.20,71.335,50.20,21.970,129.10,2024-06-29 02:05:23,63.053607,-82.022561,0.0
377,880e327767fffff,-56.605,0.0,2.50,150.6,90.670,36.65,55.435,20.00,38.055,69.15,17.685,127.80,2024-07-04 19:24:13,63.043066,-82.020684,0.0
378,880e327769fffff,-55.295,0.0,2.50,150.6,93.545,59.35,31.480,21.40,59.000,61.85,17.355,128.65,2024-07-04 20:52:51,63.058982,-82.002637,0.0


In [75]:
df_points['val'] = [0 for i in range(len(df_points['h3_res']))]

In [76]:
# add geometry

from shapely.geometry import Polygon

def add_geometry(row):
    # print(row)
    points = h3.h3_to_geo_boundary(row['h3_res'], True)
    # print(Polygon(points))
    return Polygon(points)

df_points['geometry'] = (df_points.apply(add_geometry,axis=1))

In [77]:
# Function to generate geojson objects

from geojson import Feature, Point, FeatureCollection, Polygon
import json

def hexagons_dataframe_to_geojson(df_hex, hex_id_field,geometry_field, value_field,file_output = None):

    list_features = []
    print(df_hex)

    for i, row in df_hex.iterrows():
        print(row)
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else:
      return feat_collection

In [78]:
geojson_obj = (hexagons_dataframe_to_geojson
                (df_points,
                 hex_id_field='h3_res',
                 value_field='nasc0',
                 geometry_field='geometry'))

              h3_res  bottom_hardness  bottom_roughness  wave_depth  depth  \
0    880e320001fffff          -22.700               1.0        4.50  133.9   
1    880e320003fffff          -22.255               1.2        5.20  128.5   
2    880e320005fffff          -49.225               0.0        4.50  150.6   
3    880e320007fffff          -22.460               0.9        6.00  127.2   
4    880e320009fffff          -23.960               0.9        4.80  137.8   
..               ...              ...               ...         ...    ...   
375  880e327763fffff          -59.395               0.0        2.10  150.6   
376  880e327765fffff          -56.870               0.0        2.70  150.6   
377  880e327767fffff          -56.605               0.0        2.50  150.6   
378  880e327769fffff          -55.295               0.0        2.50  150.6   
379  880e32776dfffff          -58.535               0.0        2.45  150.6   

       nasc0  fish_depth0   nasc1  fish_depth1   nasc2  fish_de

In [79]:
df

,bottom_hardness,bottom_roughness,wave_depth,depth,nasc0,fish_depth0,nasc1,fish_depth1,nasc2,fish_depth2,nasc3,fish_depth3,Datetime,Lat,Long,val,h3_res
0,-60.51,0.0,2.1,150.6,75.97,118.1,68.85,121.3,52.31,126.7,53.76,126.4,2024-06-26 20:46:10,62.947980,-82.031340,0,880e3275c5fffff
1,-61.16,0.0,2.1,150.6,70.10,115.3,63.27,118.1,45.54,126.1,46.67,125.8,2024-06-26 20:46:14,62.947980,-82.031340,0,880e3275c5fffff
2,-61.05,0.0,2.1,150.6,84.39,115.4,76.96,118.4,53.72,126.6,55.18,126.3,2024-06-26 20:46:16,62.947980,-82.031340,0,880e3275c5fffff
3,-55.47,0.0,2.1,150.6,91.19,99.2,83.91,112.1,48.22,126.4,49.46,126.3,2024-06-26 20:46:18,62.947980,-82.031340,0,880e3275c5fffff
4,-56.92,0.0,2.1,150.6,100.78,109.6,93.15,114.0,55.86,126.8,57.40,126.5,2024-06-26 20:46:20,62.947980,-82.031340,0,880e3275c5fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202850,-9.60,0.2,2.1,59.7,16.59,18.2,14.16,14.2,2.43,52.6,0.00,0.0,2024-07-06 14:17:00,62.993860,-82.024360,0,880e32755dfffff
202851,-7.59,0.2,2.1,59.6,29.25,27.7,16.54,12.9,12.71,58.3,0.00,0.0,2024-07-06 14:17:02,62.993848,-82.024353,0,880e32755dfffff
202852,-9.30,0.2,4.8,59.5,22.44,32.5,11.71,21.2,10.73,58.2,0.00,0.0,2024-07-06 14:17:06,62.993823,-82.024338,0,880e32755dfffff
202853,-6.02,0.2,2.7,59.4,43.13,16.8,29.54,6.9,13.58,57.8,0.00,0.0,2024-07-06 14:17:08,62.993811,-82.024331,0,880e32755dfffff


In [85]:

import plotly.express as px

fig = (px.choropleth_mapbox(
                    df_points, 
                    geojson=geojson_obj, 
                    locations='h3_res', 
                    color='depth',
                    color_continuous_scale="Viridis",
                    range_color=(0,-df_points['depth'].mean()+100),
                    mapbox_style='carto-positron',
                    zoom=7,
                    center = {"lat": 62.1023, "lon": -83.54},
                    opacity=0.5,
                    #labels={'Kommuner':'Peak Power Kommuner'}
))

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(autosize=False,width=1000,height=1200)
fig.show()